langchain-core: The base abstractions (like runnable, messages, and standard interfaces).

langchain: The chain, agent, and retrieval logic.

langchain-community: The massive collection of specific tools, document loaders, vector stores, and chat models provided by the community and third parties (e.g., Wikipedia tools, FAISS vector store, specific PDF loaders).

We will be taking an article draft and using LangChain to generate various useful items around this article. We'll be creating:

01. An article title
02. An article description
03. Editor advice where we will insert an additional paragraph in the article
04. A thumbnail / hero image for our article.

Here we input our article to start with. Currently this is using an article from the Aurelio AI learning page.


In [23]:
import os
from langchain_openai import ChatOpenAI
import textwrap

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
MODEL = "qwen3:0.6b" # okay
# MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [24]:
llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.0)

creative_llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.9)

# response = model.invoke("Why do parrots talk?")
# response.content

In [25]:
article = """
\
We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.

With agents, we allow LLMs to integrate with code — allowing AI to search the web, perform math, and essentially integrate into anything we can build with code. It should be clear the scope of use cases is phenomenal where AI can integrate with the broader world of software.

In this introduction to AI agents, we will cover the essential concepts that make them what they are and why that will make them the core of real-world AI in the years to come.

---

## Neuro-Symbolic Systems

Neuro-symbolic systems consist of both neural and symbolic computation, where:

- Neural refers to LLMs, embedding models, or other neural network-based models.
- Symbolic refers to logic containing symbolic logic, such as code.

Both neural and symbolic AI originate from the early philosophical approaches to AI: connectionism (now neural) and symbolism. Symbolic AI is the more traditional AI. Diehard symbolists believed they could achieve true AGI via written rules, ontologies, and other logical functions.

The other camp were the connectionists. Connectionism emerged in 1943 with a theoretical neural circuit but truly kicked off with Rosenblatt's perceptron paper in 1958 [1][2]. Both of these approaches to AI are fascinating but deserve more time than we can give them here, so we will leave further exploration of these concepts for a future chapter.

Most important to us is understanding where symbolic logic outperforms neural-based compute and vice-versa.

| Neural | Symbolic |
| --- | --- |
| Flexible, learned logic that can cover a huge range of potential scenarios. | Mostly hand-written rules which can be very granular and fine-tuned but hard to scale. |
| Hard to interpret why a neural system does what it does. Very difficult or even impossible to predict behavior. | Rules are written and can be understood. When unsure why a particular ouput was produced we can look at the rules / logic to understand. |
| Requires huge amount of data and compute to train state-of-the-art neural models, making it hard to add new abilities or update with new information. | Code is relatively cheap to write, it can be updated with new features easily, and latest information can often be added often instantaneously. |
| When trained on broad datasets can often lack performance when exposed to unique scenarios that are not well represented in the training data. | Easily customized to unique scenarios. |
| Struggles with complex computations such as mathematical operations. | Perform complex computations very quickly and accurately. |

Pure neural architectures struggle with many seemingly simple tasks. For example, an LLM *cannot* provide an accurate answer if we ask it for today's date.

Retrieval Augmented Generation (RAG) is commonly used to provide LLMs with up-to-date knowledge on a particular subject or access to proprietary knowledge.

### Giving LLMs Superpowers

By 2020, it was becoming clear that neural AI systems could not perform tasks symbolic systems typically excelled in, such as arithmetic, accessing structured DB data, or making API calls. These tasks require discrete input parameters that allow us to process them reliably according to strict written logic.

In 2022, researchers at AI21 developed Jurassic-X, an LLM-based "neuro-symbolic architecture." Neuro-symbolic refers to merging the "neural computation" of large language models (LLMs) with more traditional (i.e. symbolic) computation of code.

Jurassic-X used the Modular Reasoning, Knowledge, and Language (MRKL) system [3]. The researchers developed MRKL to solve the limitations of LLMs, namely:

- Lack of up-to-date knowledge, whether that is the latest in AI or something as simple as today's date.
- Lack of proprietary knowledge, such as internal company docs or your calendar bookings.
- Lack of reasoning, i.e. the inability to perform operations that traditional software is good at, like running complex mathematical operations.
- Lack of ability to generalize. Back in 2022, most LLMs had to be fine-tuned to perform well in a specific domain. This problem is still present today but far less prominent as the SotA models generalize much better and, in the case of MRKL, are able to use tools relatively well (although we could certainly take the MRKL solution to improve tool use performance even today).

MRKL represents one of the earliest forms of what we would now call an agent; it is an LLM (neural computation) paired with executable code (symbolic computation).

## ReAct and Tools

There is a misconception in the broader industry that an AI agent is an LLM contained within some looping logic that can generate inputs for and execute code functions. This definition of agents originates from the huge popularity of the ReAct agent framework and the adoption of a similar structure with function/tool calling by LLM providers such as OpenAI, Anthropic, and Ollama.

![ReAct agent flow with the Reasoning-Action loop [4]. When the action chosen specifies to use a normal tool, the tool is used and the observation returned for another iteration through the Reasoning-Action loop. To return a final answer to the user the LLM must choose action "answer" and provide the natural language response, finishing the loop.](/images/posts/ai-agents/ai-agents-00.png)

<small>ReAct agent flow with the Reasoning-Action loop [4]. When the action chosen specifies to use a normal tool, the tool is used and the observation returned for another iteration through the Reasoning-Action loop. To return a final answer to the user the LLM must choose action "answer" and provide the natural language response, finishing the loop.</small>

Our "neuro-symbolic" definition is much broader but certainly does include ReAct agents and LLMs paired with tools. This agent type is the most common for now, so it's worth understanding the basic concept behind it.

The **Re**ason **Act**ion (ReAct) method encourages LLMs to generate iterative *reasoning* and *action* steps. During *reasoning,* the LLM describes what steps are to be taken to answer the user's query. Then, the LLM generates an *action,* which we parse into an input to some executable code, which we typically describe as a tool/function call.

![ReAct method. Each iteration includes a Reasoning step followed by an Action (tool call) step. The Observation is the output from the previous tool call. During the final iteration the agent calls the answer tool, meaning we generate the final answer for the user.](/images/posts/ai-agents/ai-agents-01.png)

<small>ReAct method. Each iteration includes a Reasoning step followed by an Action (tool call) step. The Observation is the output from the previous tool call. During the final iteration the agent calls the answer tool, meaning we generate the final answer for the user.</small>

Following the reason and action steps, our action tool call returns an observation. The logic returns the observation to the LLM, which is then used to generate subsequent reasoning and action steps.

The ReAct loop continues until the LLM has enough information to answer the original input. Once the LLM reaches this state, it calls a special *answer* action with the generated answer for the user.

## Not only LLMs and Tool Calls

LLMs paired with tool calling are powerful but far from the only approach to building agents. Using the definition of neuro-symbolic, we cover architectures such as:

- Multi-agent workflows that involve multiple LLM-tool (or other agent structure) combinations.
- More deterministic workflows where we may have set neural model-tool paths that may fork or merge as the use case requires.
- Embedding models that can detect user intents and decide tool-use or LLM selection-based selection in vector space.

These are just a few high-level examples of alternative agent structures. Far from being designed for niche use cases, we find these alternative options to frequently perform better than the more common ReAct or Tool agents. We will cover all of these examples and more in future chapters.

---

Agents are fundamental to the future of AI, but that doesn't mean we should expect that future to come from agents in their most popular form today. ReAct and Tool agents are great and handle many simple use cases well, but the scope of agents is much broader, and we believe thinking beyond ReAct and Tools is key to building future AI.

---

You can sign up for the [Aurelio AI newsletter](https://b0fcw9ec53w.typeform.com/to/w2BDHVK7) to stay updated on future releases in our comprehensive course on agents.

---

## References

[1] The curious case of Connectionism (2019) [https://www.degruyter.com/document/doi/10.1515/opphil-2019-0018/html](https://www.degruyter.com/document/doi/10.1515/opphil-2019-0018/html)

[2] F. Rosenblatt, [The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain](https://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf) (1958), Psychological Review

[3] E. Karpas et al. [MRKL Systems: A Modular, Neuro-Symbolic Architecture That Combines Large Language Models, External Knowledge Sources and Discrete Reasoning](https://arxiv.org/abs/2205.00445) (2022), AI21 Labs
"""

In [26]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant called {name} that helps generate article titles.",
    input_variable=["name"],
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
The article is here for you to examine:

---

 {article}
 
---
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided and need it simple and short.""",
    input_variables=["article"],
)

In [27]:
# user_prompt.format(article="TEST STRING")

In [28]:
# preview of the user prompt output. Later for the variable `article` we can give a real value.
# print(user_prompt.format(article="TEST STRING").content)

In [29]:
from langchain_core.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [30]:
# format `first_prompt` also

first_prompt.format(article="TEST STRING", name="TEST NAME")

'System: You are an AI assistant called TEST NAME that helps generate article titles.\nHuman: You are tasked with creating a name for a article.\nThe article is here for you to examine:\n\n---\n\n TEST STRING\n\n---\nThe name should be based of the context of the article.\nBe creative, but make sure the names are clear, catchy,\nand relevant to the theme of the article.\n\nOnly output the article name, no other explanation or\ntext can be provided and need it simple and short.'

In [31]:
print(first_prompt.format(article="TEST STRING", name="TEST NAME"))

System: You are an AI assistant called TEST NAME that helps generate article titles.
Human: You are tasked with creating a name for a article.
The article is here for you to examine:

---

 TEST STRING

---
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided and need it simple and short.


In [32]:
chain_one = (
    {
        "article": lambda x: x["article"],
        "name": lambda x: x["name"],
    }
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content} # extract the content field of llm output
)



# {left side} | {right side} => left is going to go into the right side
# {...} go into `first_prompt`, first prompt go into `creative_llm`, creative llm go into {...}

article_title_msg = chain_one.invoke({
    "article" : article,
    "name" : "Title Finder"
})

article_title_msg

{'article_title': 'Neuro-Symbolic Agents: Bridging the Gap in AI Innovation.'}

In [11]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps build good articles.",
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a description for
the article. The article is here for you to examine:

---

{article}

---

Here is the article title '{article_title}'.

Output the SEO friendly article description. 
make sure we don't exceed 120 characters.
Do not output anything other than the description.""",
    input_variables=["article", "article_title"],
)

second_prompt = ChatPromptTemplate.from_messages([system_prompt, second_user_prompt])


chain_two = (
    {
        "article": lambda x: x["article"], 
        "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | llm
    | {"summery": lambda x: x.content}
)

article_description_msg = chain_two.invoke (
    {
        "article": article,
        "article_title": article_title_msg["article_title"]
    }
)

article_description_msg

{'summery': 'Neuro-symbolic agents for the future of AI: explore how combining neural and symbolic systems will shape the next generation of AI. With applications in search, computation, and integration, these agents will redefine the boundaries of real-world AI. The MRKL architecture, ReAct method, and future use cases are highlighted. [1-3]'}

In [12]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a new paragraph for the
article. The article is here for you to examine:

---

{article}

---

Choose one paragraph to review and edit. During your edit
ensure you provide constructive feedback to the user so they
can learn where to improve their own writing.""",
    input_variables=["article"]
)

# prompt template 3: creating a new paragraph for the article
third_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    third_user_prompt
])


#* force a Large Language Model (LLM) to output data in a strict, structured format rather than free text.
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(
        description = "Constructive feedback on the original paragraph."
    )

structured_llm = creative_llm.with_structured_output(Paragraph)


chain_tree = (
    {"article": lambda x: x["article"]}
    | third_prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

out = chain_tree.invoke({"article": article})
out

{'original_paragraph': "We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.",
 'edited_paragraph': "We believe AI's short-term to the mid-term future belongs to agents, and that the long-term future of *AGI* may evolve through agentic systems. Our definition of agents includes any neuro-symbolic systems that integrate neural AI (like large language models) with traditional software. These systems enable AI to perform a wide array of tasks, from conducting complex computations to interacting with external knowledge bases.",
 'feedback': "The paragraph highlights the importance of agents for the future, making it a strong start. You can expand on the term 'short-term' to better emphasize the focus on current and future systems. The definition could be more precise, emphasizing 

## image generation

use a model that can generate images (like open ai dall-e-3)

In [16]:
# from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
# from langchain_core.prompts import PromptTemplate

# image_prompt = PromptTemplate(
#     input_variables=["article"],
#     template=(
#         "Generate a prompt with less then 500 characters to generate an image "
#         "based on the following article: {article}"
#     ),
# )


# from skimage import io
# import matplotlib.pyplot as plt
# from langchain_core.runnables import RunnableLambda


# def generate_and_display_image(image_prompt):
#     image_url = DallEAPIWrapper(model="dall-e-3").run(image_prompt)
#     image_data = io.imread(image_url)

#     # And update the display code to:
#     plt.imshow(image_data)
#     plt.axis("off")
#     plt.show()


# # we wrap this in a RunnableLambda for use with LCEL
# image_gen_runnable = RunnableLambda(generate_and_display_image)


# # chain 4: inputs: article, article_para / outputs: new_suggestion_article
# chain_four = (
#     {"article": lambda x: x["article"]}
#     | image_prompt
#     | llm
#     | (lambda x: x.content)
#     | image_gen_runnable
# )


# chain_four.invoke({"article": article})

# Langsmith

---

In [ ]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or getpass(
    "Enter LangSmith API Key: "
)

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "langchain_course_1"

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
MODEL = "qwen3:0.6b"  # okay
# MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [3]:
# testing

import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

llm = ChatOpenAI(temperature=0.0, model=MODEL, base_url=BASE_URL)

llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 11, 'total_tokens': 104, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen3:0.6b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-150', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b5b26-b02c-76f3-8324-fae4de46a2ba-0', usage_metadata={'input_tokens': 11, 'output_tokens': 93, 'total_tokens': 104, 'input_token_details': {}, 'output_token_details': {}})

### Tracing Non-LangChain Code

In [4]:
from langsmith import traceable
import random
import time


@traceable
def generate_random_number():
    return random.randint(0, 100)


@traceable
def generate_string_delay(input_str: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_str} ({number})"


@traceable
def random_error():
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "No error"

In [5]:
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("Hello")
    try:
        random_error()
    except ValueError:
        pass

  0%|          | 0/10 [00:00<?, ?it/s]

#### with specific names

In [6]:
from langsmith import traceable


@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "I'm great how are you?"

In [7]:
for _ in tqdm(range(5)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

  0%|          | 0/5 [00:00<?, ?it/s]

---

# Prompting technique

We'll start by looking at the various parts of our prompt. For RAG use-cases we'll typically have three core components however this is _very_ use-cases dependant and can vary significantly. Nonetheless, for RAG we will typically see:

* **Rules for our LLM**: this part of the prompt sets up the behavior of our LLM, how it should approach responding to user queries, and simply providing as much information as possible about what we're wanting to do as possible. We typically place this within the _system prompt_ of an chat LLM.

* **Context**: this part is RAG-specific. The context refers to some _external information_ that we may have retrieved from a web search, database query, or often a _vector database_. This external information is the **R**etrieval **A**ugmentation part of **RA**G. For chat LLMs we'll typically place this inside the chat messages between the assistant and user.

* **Question**: this is the input from our user. In the vast majority of cases the question/query/user input will always be provided to the LLM (and typically through a _user message_). However, the format and location of this being provided often changes.

* **Answer**: this is the answer from our assistant, again this is _very_ typical and we'd expect this with every use-case.

The below is an example of how a RAG prompt may look:

```
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
```

In [39]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "none"
# MODEL = "functiongemma:270m" # still no good
MODEL = "qwen3:0.6b"  # okay
# MODEL = "gemma3:4b" # okay
BASE_URL = "http://localhost:11434/v1"

In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""


# from langchain.prompts import (
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate
# )

# prompt_template = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_template(prompt),
#     HumanMessagePromptTemplate.from_template("{query}"),
# ])

# short way
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        ("user", "{query}"), # now langchain can automatically detects inputs. so no need to specify input_variables.
    ]
)

In [41]:
prompt_template.input_variables

['context', 'query']

In [42]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, temperature=0.0)

# pipeline = (
#     {
#         "query": lambda x: x["query"], # to these can add (x["query"].lower())
#         "context": lambda x: x["context"]
#     }
#     | prompt_template
#     | llm
# )

pipeline = prompt_template | llm #can use like this also

context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [48]:
prompt_template.format( query = "___QUERY___", context = "___CONTEXT___")

'System: \nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: ___CONTEXT___\n\nHuman: ___QUERY___'

In [ ]:
response = pipeline.invoke({"query": query, "context": context})
response.content

'Aurelio AI develops tooling for AI engineers, focusing on language AI with expertise in building AI agents and information retrieval. They offer open-source frameworks like Semantic Router and Semantic Chunkers, an AI Platform for engineers to build AI tools, and development services to other organizations. Additionally, they became LangChain Experts in September 2024 after a long track record of delivering AI solutions built with the LangChain ecosystem.'

`video time : 1:12:40`